## Import libraries and Load data

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Load data
!pip install gdown
!gdown --folder 1EOi1JmFkTPege1geI0f9JmXfEk3kZkGS

df_paymnent = pd.read_csv("/content/E-Wallet Dataset/payment_report.csv")
df_product = pd.read_csv("/content/E-Wallet Dataset/product.csv")
df_transaction = pd.read_csv("/content/E-Wallet Dataset/transactions.csv")

Retrieving folder contents
Processing file 1RFsKyqTpVuXU7n3J8lh2dIddDKdgkhr6 payment_report.csv
Processing file 1IRwi4d-aR7N8FgezoIlLzT5W8tRMtAuP product.csv
Processing file 1MLeZHyjrP7cx1K9MWb3zqWxW1FvBPDiG transactions.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1RFsKyqTpVuXU7n3J8lh2dIddDKdgkhr6
To: /content/E-Wallet Dataset/payment_report.csv
100% 30.3k/30.3k [00:00<00:00, 76.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IRwi4d-aR7N8FgezoIlLzT5W8tRMtAuP
To: /content/E-Wallet Dataset/product.csv
100% 8.61k/8.61k [00:00<00:00, 39.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MLeZHyjrP7cx1K9MWb3zqWxW1FvBPDiG
To: /content/E-Wallet Dataset/transactions.csv
100% 78.2M/78.2M [00:00<00:00, 305MB/s]
Download completed


# Explore Data Analysis (EDA)

## EDA payment_enriched
- There are 22 missing values in category & team_own columns.
- There are no duplicated values.

In [3]:
# Merge payment_report.csv with product.csv
df_payment_enriched = pd.merge(df_paymnent, df_product, how = "left", on = "product_id")
df_payment_enriched.head()

,report_month,payment_group,product_id,source_id,volume,category,team_own
0,2023-01,payment,12,45,624110375,PXXXXXT,ASD
1,2023-01,payment,17,45,335715113,PXXXXXB,ASD
2,2023-01,payment,18,45,737784466,PXXXXXB,ASD
3,2023-01,payment,19,45,120963069,PXXXXXM2,ASD
4,2023-01,payment,20,45,319653158,PXXXXXB,ASD


In [4]:
# Check general info df_payment_enriched
df_payment_enriched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   report_month   919 non-null    object
 1   payment_group  919 non-null    object
 2   product_id     919 non-null    int64 
 3   source_id      919 non-null    int64 
 4   volume         919 non-null    int64 
 5   category       897 non-null    object
 6   team_own       897 non-null    object
dtypes: int64(3), object(4)
memory usage: 50.4+ KB


In [5]:
# Check duplicates
df_payment_enriched.duplicated().sum()

np.int64(0)

## EDA transaction
- There are 49,059 missing values for sender_id, 164,795 missing values for receiver_id and 1,317,907 missing values for extra_info.
- There are 28 duplicated values.

### Action:
- Fill missing values in sender_id, receiver_id with “Unknown”.
- Remove extra_info column which is not important for transaction and payment analysis.
- Remove 28 duplicated values.


In [6]:
# Check general info df_transaction
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324002 entries, 0 to 1324001
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transaction_id  1324002 non-null  int64  
 1   merchant_id     1324002 non-null  int64  
 2   volume          1324002 non-null  int64  
 3   transType       1324002 non-null  int64  
 4   transStatus     1324002 non-null  int64  
 5   sender_id       1274943 non-null  float64
 6   receiver_id     1159207 non-null  float64
 7   extra_info      6095 non-null     object 
 8   timeStamp       1324002 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 90.9+ MB


In [7]:
# Handle missing values
df_transaction['sender_id'] = df_transaction['sender_id'].fillna(value='Unknown')
df_transaction['receiver_id'] = df_transaction['receiver_id'].fillna(value='Unknown')
df_transaction.drop('extra_info', axis=1, inplace=True)

In [8]:
# Check duplicates
df_transaction.duplicated().sum()

np.int64(28)

In [9]:
# Remove duplicates
df_transaction = df_transaction.drop_duplicates()

df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1323974 entries, 0 to 1324001
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   transaction_id  1323974 non-null  int64 
 1   merchant_id     1323974 non-null  int64 
 2   volume          1323974 non-null  int64 
 3   transType       1323974 non-null  int64 
 4   transStatus     1323974 non-null  int64 
 5   sender_id       1323974 non-null  object
 6   receiver_id     1323974 non-null  object
 7   timeStamp       1323974 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 90.9+ MB


# Transaction & Payment Analysis

## 1. Top 3 product_ids with the highest volume.

In [10]:
df_payment_enriched.groupby("product_id", as_index=False).agg({"volume":"sum"}).sort_values("volume", ascending = False).head(3)

,product_id,volume
279,1976,61797583647
43,429,14667676567
38,372,13713658515


--> Top 3 product_ids with the highest volume: 1976, 429, 372.

## 2. Given that 1 product_id is only owed by 1 team, are there any abnormal products against this rule?

In [11]:
df_abnormal = df_payment_enriched.groupby("product_id", as_index=False)["team_own"].nunique()
df_abnormal[df_abnormal["team_own"] != 1]

,product_id,team_own
0,3,0
279,1976,0
308,10033,0


--> There are 3 abnormal products (3, 1976, 10033) against the rule. These products ID are not owned by any team.

## 3. Find the team has had the lowest performance (lowest volume) since Q2.2023. Find the category that contributes the least to that team.

In [12]:
df_q2 = df_payment_enriched[df_payment_enriched["report_month"] >= "2023-04"]
df_q2.groupby("team_own", as_index=False).agg({"volume":"sum"}).sort_values("volume")

,team_own,volume
0,APS,51141753
2,ASL,11284361730
1,ASD,31090428473


--> The team had the lowest performance in Q2.2023 is Team APS.

In [13]:
df_q2[df_q2["team_own"] == "APS"].groupby("category", as_index=False).agg({"volume":"sum"}).sort_values("volume")

,category,volume
0,PXXXXXE,25232438
1,PXXXXXS,25909315


--> The category contributes the least to APS is PXXXXXE.

## 4. Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?

In [14]:
df_refund = df_payment_enriched[df_payment_enriched["payment_group"] == "refund"]
df_contribute = df_refund.groupby("source_id", as_index=False).agg({"volume":"sum"}).sort_values("volume", ascending=False)
df_contribute['%contribute'] = df_contribute['volume'] / df_contribute['volume'].sum() * 100
df_contribute

,source_id,volume,%contribute
1,38,36527454759,59.108225
2,39,16119059662,26.083641
0,37,9151069226,14.808134


--> The source_id with the highest contribution is 38 (contributes 59.10%).

## 5. Define type of transactions (‘transaction_type’) for each row, given:
- transType = 2 & merchant_id = 1205: Bank Transfer Transaction
- transType = 2 & merchant_id = 2260: Withdraw Money Transaction
- transType = 2 & merchant_id = 2270: Top Up Money Transaction
- transType = 2 & others merchant_id: Payment Transaction
- transType = 8, merchant_id = 2250: Transfer Money Transaction
- transType = 8 & others merchant_id: Split Bill Transaction
- Remained cases are invalid transactions

In [15]:
conditions = [(df_transaction["transType"] == 2) & (df_transaction["merchant_id"] == 1205),
              (df_transaction["transType"] == 2) & (df_transaction["merchant_id"] == 2260),
              (df_transaction["transType"] == 2) & (df_transaction["merchant_id"] == 2270),
              (df_transaction["transType"] == 2),
              (df_transaction["transType"] == 8) & (df_transaction["merchant_id"] == 2250),
              (df_transaction["transType"] == 8),
              ]

results = ["Bank Transfer Transaction",
          "Withdraw Money Transaction",
          "Top Up Money Transaction",
          "Payment Transaction",
          "Transfer Money Transaction",
          "Split Bill Transaction",
          ]

df_transaction["transaction_type"] = np.select(conditions,
                                               results,
                                               default="Invalid Transaction")
df_transaction

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,timeStamp,transaction_type
0,3002692434,5,100000,24,1,10199794.0,199794.0,1682932054455,Invalid Transaction
1,3002692437,305,20000,2,1,14022211.0,14022211.0,1682932054912,Payment Transaction
2,3001960110,7255,48605,22,1,Unknown,10530940.0,1682932055000,Invalid Transaction
3,3002680710,2270,1500000,2,1,10059206.0,59206.0,1682932055622,Top Up Money Transaction
4,3002680713,2275,90000,2,1,10004711.0,4711.0,1682932056197,Payment Transaction
...,...,...,...,...,...,...,...,...,...
1323997,3003723030,305,20000,2,1,24524311.0,Unknown,1683035672634,Payment Transaction
1323998,3003723033,2270,100000,2,1,10277242.0,277242.0,1683035672876,Top Up Money Transaction
1323999,3003723036,2270,100000,2,1,10144599.0,144599.0,1683035672892,Top Up Money Transaction
1324000,3003723039,5,400,22,1,10028007.0,21013762.0,1683035672896,Invalid Transaction


## 6. Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers.

In [16]:
df_transaction[df_transaction["transaction_type"] != "Invalid Transaction"].groupby("transaction_type", as_index=False).agg({"transaction_id":"nunique",
                                                                                                                              "volume":"sum",
                                                                                                                              "sender_id":"nunique",
                                                                                                                              "receiver_id":"nunique"}).sort_values("transaction_id")

,transaction_type,transaction_id,volume,sender_id,receiver_id
2,Split Bill Transaction,1376,4901464,1323,572
5,Withdraw Money Transaction,33725,23418181420,24814,24814
0,Bank Transfer Transaction,37879,50605806190,23156,9272
3,Top Up Money Transaction,290498,108605618829,110409,110409
4,Transfer Money Transaction,341173,37032880492,39021,34585
1,Payment Transaction,398665,71850608441,139583,113299
